# Set up

In [2609]:
from worde4mde import load_embeddings
sgram_mde = load_embeddings('sgram-mde')

In [2610]:
word = 'ecore'
sgram_mde[word]

array([ 0.14674647,  0.42704162,  0.17717203,  0.05179158,  0.38020504,
       -0.00091264,  0.02094162,  0.00924711, -0.13806985,  0.1403339 ,
       -0.0953936 , -0.03681302,  0.38122246, -0.03746711, -0.17595871,
       -0.02031669,  0.23675247,  0.27676958, -0.07487358, -0.1987225 ,
        0.02015282,  0.05147164, -0.223504  ,  0.00699752, -0.06145973,
        0.00920826,  0.01364714,  0.10218145, -0.25330105, -0.14080754,
        0.2541444 ,  0.4577827 , -0.16825216, -0.08050938,  0.11404785,
       -0.38822263, -0.06395938, -0.4724689 , -0.00111705,  0.27869806,
        0.28032073,  0.02976846, -0.01675524,  0.09774698,  0.18298927,
       -0.24398401, -0.3238438 ,  0.01912955,  0.11861664,  0.08640955,
        0.18799016, -0.06891091, -0.03893414, -0.04862301,  0.02151174,
       -0.39205688, -0.23195332, -0.09532862, -0.01324714, -0.24555108,
        0.09372689, -0.21971853, -0.26171875, -0.3605202 , -0.07306654,
       -0.16288763,  0.12160088,  0.11416717,  0.17378834, -0.04

In [2611]:
# import pickle

import ast
import pandas as pd
import numpy as np

from worde4mde import load_embeddings
import re

import os
import openai
from openai import OpenAI
import json

import networkx as nx
import sys


In [2612]:
from dotenv import load_dotenv

# %%

load_dotenv()

openai_key = os.getenv('openai_key')
os.environ['OPENAI_API_KEY'] = openai_key

## Input

In [2613]:
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######
##### update here to change input file directory ######

instructor_dir = "results/"

# 01_LabTracker
# 02_CelO
# 03_TSS
# 04_SHAS
# 05_OTS
# 06_Block
# 07_Tile
# 08_HBMS
problem_id = "08_HBMS"

# location of reference domain model is stored. It is in .txt format
instructor_input = instructor_dir + problem_id + ".txt"

# gpt-4o-mini
# gpt-4
# gpt-4o
llm_id = "gpt-4o"

# yujing
# sherpa
method_id = "yujing"

student_dir = instructor_dir + llm_id + '/' + method_id + '/' + problem_id + '/'
if not os.path.exists(student_dir):
    # Create the directory
    os.makedirs(student_dir)
    
# location of generated domain model is saved. The file should be in .txt    
student_input = student_dir + problem_id + ".txt"
student_input
student_out_dir = student_dir + "result/"
student_out_dir

'results/gpt-4o/yujing/07_Tile/result/'

In [2614]:
# First of all, you need to load the embeddings (currently supported: 'sgram-mde' and 'glove-mde')
sgram_mde = load_embeddings('sgram-mde')
# glove = load_embeddings('glove-mde')
# sgram_mde["id"]

In [2615]:
# word = 'id'
# sgram_mde.most_similar(positive=[word])

In [2616]:
def inEmbedding(word, embedding):
  try:
    return True, embedding[word]
  except:
    return False, None

In [2617]:

def splitCamelCase(word):
  splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', word)).split()
  return splitted

splitCamelCase("device ID")

['device', 'ID']

In [2618]:
splitCamelCase("CommandStatus")

['Command', 'Status']

In [2619]:
# client = OpenAI()

# def run_llm(prompt, model ="gpt-3.5-turbo"):
#   response = client.chat.completions.create(
#     model=model,
#     messages=[
#       {"role": "user", "content": prompt},

#     ]
#   )
#   return response.choices[0].message.content

In [2620]:
client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [2621]:
def get_all_info(class_index, class_nodes, list_of_classes, list_edges, need_edge = True):
  result = ''
  result +=  list_of_classes[class_index] + "\n"
  node = class_nodes[class_index]
  if "abstract" in node:
      node = node.replace("abstract", "").strip()
  if "Abstract" in node:
      node = node.replace("Abstract", "").strip()
  if need_edge:
    for edge in list_edges:
      element = [i.strip() for i in edge.split()]
      if node in element:
        result += edge + "\n"

  return  result

In [2622]:
def match_classes(raw_1, raw_2, dict_attr, thresh = 0.5, verbose = False):
  # raw_1: nodes from reference solution
  # raw_2: nodes from student solution
  # dict_attr: dictionary for embeddings
  # thresh for subst cost
  # map class lists raw_1 and raw_2
  print(dict_attr)
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < thresh:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]
    


  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

# Group

In [2623]:
class Grader():
  def __init__(self):
    self.ref = None
    self.stu = None

In [2624]:
grader = Grader()

## Class

### Define metadata

In [2625]:
class ModelInfo():

  def __init(self):
    self.cls_atr = {}
    self.cls_name = []
    self.raw_dsl = []
    self.rel = None

In [2626]:
ref_cls = ModelInfo()
stu_cls = ModelInfo()

In [2627]:
grader.ref = ref_cls
grader.stu = stu_cls

In [2628]:
with open(instructor_input, 'r') as file:
    # Read the entire content of the file
    instructor_all = file.read()

with open(student_input, 'r') as file:
    # Read the entire content of the file
    student_all = file.read()

In [2629]:
print(instructor_all)

Enumerations:
Mode(design, game)
Color(red, blue, green, yellow)

Classes:
TileO()
Game(int currentConnectionPieces, Mode mode, int spaceConnectionPieces, int numberOfActionCards)


Connection()

Die()
Deck()

abstract Tile(int x, int y, boolean hasBeenVisited)
abstract ActionCard (string instructions)
RollDieActionCard()
ConnectTilesActionCard()
RemoveConnectionActionCard()
TeleportActionCard()
LoseTurnActionCard()
WinTile()
ActionTile(int inactivityPeriod, int turnsUntilActive)
NormalTile()
Player(int number, int turnsUntilActive, Color color)


Relationships:



In [2630]:
print(student_all)

Enumerations:
ActionCardType(RollExtraTurn, ConnectTiles, RemoveConnection, MoveToTile, LoseTurn)

Classes:
Game(int currentPlayerIndex, boolean isGameOver, string version, string status, string designerName)
Board(int width, int height)



Die(int sides)

Tile(boolean isVisited, boolean isHidden, boolean isActionTile, int turnsRemaining)
ActionCard(ActionCardType type, string description)


Player(string name, string color, int currentPosition)


Relationships:



In [2631]:
shas_sol = instructor_all.split("Relationships:")[0]
stu_sol_g =  student_all.split("Relationships:")[0] # student solution

In [2632]:
ref_classes_raw = []
tmp = shas_sol.strip().splitlines()
ref_classes_raw = [i.strip() for i in tmp if len(i) > 0]
ref_classes_raw

reference_class = []
tmp = shas_sol.strip().splitlines()
reference_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]

In [2633]:
stu_classes_raw = []
tmp = stu_sol_g.strip().splitlines()
stu_classes_raw = [i.strip() for i in tmp if len(i) > 0]
stu_classes_raw

tmp = stu_sol_g.strip().splitlines()
stu_class = [i.split("(")[0].strip() for i in tmp if len(i) > 0]
stu_class

['Enumerations:',
 'ActionCardType',
 'Classes:',
 'Game',
 'Board',
 'Die',
 'Tile',
 'ActionCard',
 'Player']

In [2634]:
ref_attributes = {}
enum_index = ref_classes_raw.index("Enumerations:")
regular_index = ref_classes_raw.index("Classes:")
for cla, dsl in zip(reference_class, ref_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = ref_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"
    ref_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        ref_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

In [2635]:
reference_class.remove("Enumerations:")
reference_class.remove("Classes:")

ref_classes_raw.remove("Enumerations:")
ref_classes_raw.remove("Classes:")

In [2636]:
ref_cls.cls_name = reference_class
ref_cls.raw_dsl = ref_classes_raw
ref_cls.cls_atr = ref_attributes

In [2637]:
stu_attributes = {}
enum_index = stu_classes_raw.index("Enumerations:")
regular_index = stu_classes_raw.index("Classes:")

for cla, dsl in zip(stu_class, stu_classes_raw):
  if dsl == "Enumerations:" or dsl == "Classes:":
    continue
  else:
    index = stu_classes_raw.index(dsl)
    if index > regular_index:
      class_type = "regular"
      if "abstract" in dsl:
        class_type = "abstract"
    else:
      class_type = "enum"

    stu_attributes[cla] = {"score": 0, "type":class_type, "dsl":dsl, "counterpart": None, "attributes": {}}

    print(dsl)
    attributes = dsl.split("(")[1][:-1].split(",")
    for attr in attributes:
      attr = attr.strip()
      if len(attr) > 0:
        stu_attributes[cla]["attributes"][attr] = {"score":0, "counterpart": None}

ActionCardType(RollExtraTurn, ConnectTiles, RemoveConnection, MoveToTile, LoseTurn)
Game(int currentPlayerIndex, boolean isGameOver, string version, string status, string designerName)
Board(int width, int height)
Die(int sides)
Tile(boolean isVisited, boolean isHidden, boolean isActionTile, int turnsRemaining)
ActionCard(ActionCardType type, string description)
Player(string name, string color, int currentPosition)


In [2638]:
stu_class.remove("Enumerations:")
stu_class.remove("Classes:")

stu_classes_raw.remove("Enumerations:")
stu_classes_raw.remove("Classes:")

In [2639]:
stu_cls.cls_name = stu_class
stu_cls.raw_dsl = stu_classes_raw
stu_cls.cls_atr = stu_attributes

## Edges

### reference and student solutions

In [2640]:
shas_edges = instructor_all.split("Relationships:")[1]
shas_G2_edges =  student_all.split("Relationships:")[1] # student solution

In [2641]:
shas_edges

'\n'

In [2642]:
shas_edges

'\n'

In [2643]:
shas_edges = shas_edges.replace("0..*","*")

In [2644]:
shas_G2_edges = shas_G2_edges.replace("0..*","*")

In [2645]:
tmp = shas_edges.strip().splitlines()
ref_edges = [i.strip() for i in tmp if len(i) > 0]

for i in ref_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Reference Length error:", i)

tmp = shas_G2_edges.strip().splitlines()
stu_edges = [i.strip() for i in tmp if len(i) > 0]

for i in stu_edges:
  length = len(i.split())
  if length == 5 or length == 3:
    pass
  else:
    print("Student Length error:", i)



In [2646]:
class Relationship:
  def __init__(self):
    self.rels = [] # list of dict{}
    self.raw_dsl = []
  # find first non-matched or matched reltionshipi given dsl
  def find_relation(self, dsl, matched = False):
    for index, i in enumerate(self.rels):
      if i["dsl"] == dsl and i["counterpart"] is None:
        return index, i

    return None

In [2647]:
ref_edges_obj = Relationship()
stu_edges_obj = Relationship()


ref_edges_dict = []
for e in ref_edges:
  ref_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

ref_edges_obj.rels = ref_edges_dict
ref_edges_obj.raw_dsl = ref_edges

stu_edges_dict = []
for e in stu_edges:
  stu_edges_dict.append({"dsl":e,"score":0, "counterpart": None})

stu_edges_obj.rels = stu_edges_dict
stu_edges_obj.raw_dsl = stu_edges

In [2648]:
# link to classInfo object
ref_cls.rel = ref_edges_obj
stu_cls.rel = stu_edges_obj

In [2649]:
edges = [ref_edges_obj, stu_edges_obj]

## Getting embedding

In [2650]:
def removeChars(text, chars):
    for c in chars:
        text = text.replace(c,"")
    return text


In [2651]:
def get_mde_embedding(text, embedding):
  # >>> get_mde_embedding("WhatDevice", sgram_mde)
  words = splitCamelCase(removeChars(text, ["(",")",","]))
  lowercase_list = [s.lower() for s in words]
  counter = 0
  emb = np.zeros(300)
  for w in lowercase_list:
    try:
      emb += embedding[w]
      counter += 1
    except:
      # if failed in emb the complete word, embed the token
      tmp = np.zeros(300)
      for char in list(w):

        tmp += embedding[char]
      emb += (tmp / len(w))
      counter += 1
  return emb / counter

In [2652]:
text = "AutomationStatus (Created, Edited, Activated, Deactivated)"
words = splitCamelCase(removeChars(text, ["(",")",","]))
words

['Automation', 'Status', 'Created', 'Edited', 'Activated', 'Deactivated']

In [2653]:
def cosine_distance(emb_i, emb_j):
  return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

## Stage 1.1 Class name mapping

In [2654]:
# get name embedding
# all info
similarity_mde = []
mde_embedding = sgram_mde
threshold = 0.7
similarity_mde_dsl = []
percentage = 0.8
for index, node in enumerate(ref_cls.cls_name):
  cls = node.split()[-1].strip() # get the class name, remove abstract key word
  
  mde_emb_i = get_mde_embedding(cls, mde_embedding)
  mde_emb_i_dsl = get_mde_embedding(ref_cls.cls_atr[node]["dsl"], mde_embedding)

  pair = []
  mde_pair = []
  for j, stu_node in enumerate(stu_cls.cls_name):
    cls = stu_node.split()[-1].strip() # get the class name, remove abstract key word
    print()
    mde_emb_j = get_mde_embedding(cls, mde_embedding)
    mde_emb_j_dsl = get_mde_embedding(stu_cls.cls_atr[stu_node]["dsl"], mde_embedding)

    mde_sim = cosine_distance(mde_emb_i, mde_emb_j)
    mde_sim_dsl = cosine_distance(mde_emb_i_dsl, mde_emb_j_dsl)

    mde_pair.append(mde_sim * percentage + (1-percentage) * mde_sim_dsl)

  # apply third quartile
  similarity_mde.append(mde_pair)

dict_sim_word = {}
for i in range(len(ref_cls.cls_name)):
  dict_sim_word[ref_cls.cls_name[i]] = {}
  for j in range(len(stu_cls.cls_name)):
    dict_sim_word[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_mde[i][j]

In [2655]:
for key in dict_sim_word:
    exact_match = False
    exact_cls = ""
    for cls_2 in dict_sim_word[key]:
        if dict_sim_word[key][cls_2] > 0.99:
            exact_match = True
            exact_cls = cls_2
            print(cls_2)
            break
    
    if exact_match:    # keep the exact match, others to 0
        print("update")
        for cls_2 in dict_sim_word[key]:
            if cls_2 !=  exact_cls:
                dict_sim_word[key][cls_2] = 0

similarity_mde = []
for i in dict_sim_word:
    tmp = []
    for j in dict_sim_word[i]:
        tmp.append(dict_sim_word[i][j])

    similarity_mde.append(tmp)

In [2656]:
import pandas as pd

df = pd.DataFrame(index=reference_class, columns=stu_class, data=similarity_mde)
df

,ActionCardType,Game,Board,Die,Tile,ActionCard,Player
Mode,0.267808,0.248988,0.149749,0.185278,0.149311,0.223748,0.252559
Color,0.194843,0.181019,0.146821,0.092360,0.135295,0.100038,0.236507
TileO,0.328346,0.222476,0.249976,0.181572,0.765110,0.244892,0.228274
Game,0.281891,0.931329,0.380028,0.204554,0.335641,0.306859,0.662257
Connection,0.290608,0.124347,0.118273,0.072788,0.158417,0.218217,0.139200
Die,0.104680,0.174541,0.115564,0.835174,0.049500,0.098536,0.155570
Deck,0.195565,0.172762,0.147373,0.145103,0.222717,0.177386,0.172959
abstract Tile,0.251760,0.331157,0.321670,0.107394,0.961226,0.232524,0.332810
abstract ActionCard,0.875761,0.318395,0.271694,0.130832,0.254289,0.972993,0.344441
RollDieActionCard,0.615658,0.307094,0.251918,0.574259,0.204097,0.651029,0.343016


### Stage 1.1.1 Mapping (enum / regular + abstract)

In [2657]:
ref_enum = []
ref_reg_cls = []
stu_enum = []
stu_reg_cls = []

for enum in ref_cls.cls_name:
  if ref_cls.cls_atr[enum]["type"] == "enum":
    ref_enum.append(enum)
  else:
    ref_reg_cls.append(enum)

for enum in stu_cls.cls_name:
  if stu_cls.cls_atr[enum]["type"] == "enum":
    stu_enum.append(enum)
  else:
    stu_reg_cls.append(enum)

In [2658]:
enum_mapping = match_classes(ref_enum, stu_enum, dict_sim_word, thresh = 0.5)
enum_mapping[0]

{'Mode': {'ActionCardType': 0.2678079380820224, 'Game': 0.248987697953524, 'Board': 0.14974862515593035, 'Die': 0.18527809310768195, 'Tile': 0.1493111784987161, 'ActionCard': 0.22374783803198986, 'Player': 0.2525592511102678}, 'Color': {'ActionCardType': 0.19484252897563203, 'Game': 0.1810190345394481, 'Board': 0.14682085478516266, 'Die': 0.09236025602468209, 'Tile': 0.135294549021913, 'ActionCard': 0.10003810633290375, 'Player': 0.23650691221058562}, 'TileO': {'ActionCardType': 0.32834637119773036, 'Game': 0.22247602212244316, 'Board': 0.2499757231654977, 'Die': 0.18157181031996772, 'Tile': 0.765110160655909, 'ActionCard': 0.24489214269729334, 'Player': 0.2282741324495544}, 'Game': {'ActionCardType': 0.28189102516393894, 'Game': 0.9313286889417757, 'Board': 0.3800276814784601, 'Die': 0.20455448345932425, 'Tile': 0.3356408493114902, 'ActionCard': 0.3068587291396103, 'Player': 0.6622570304271429}, 'Connection': {'ActionCardType': 0.2906075015161558, 'Game': 0.12434748342860033, 'Board':

[('Mode', None), ('Color', None), (None, 'ActionCardType')]

In [2659]:
cls_mapping = match_classes(ref_reg_cls, stu_reg_cls, dict_sim_word, thresh = 0.5)
cls_mapping[0]

{'Mode': {'ActionCardType': 0.2678079380820224, 'Game': 0.248987697953524, 'Board': 0.14974862515593035, 'Die': 0.18527809310768195, 'Tile': 0.1493111784987161, 'ActionCard': 0.22374783803198986, 'Player': 0.2525592511102678}, 'Color': {'ActionCardType': 0.19484252897563203, 'Game': 0.1810190345394481, 'Board': 0.14682085478516266, 'Die': 0.09236025602468209, 'Tile': 0.135294549021913, 'ActionCard': 0.10003810633290375, 'Player': 0.23650691221058562}, 'TileO': {'ActionCardType': 0.32834637119773036, 'Game': 0.22247602212244316, 'Board': 0.2499757231654977, 'Die': 0.18157181031996772, 'Tile': 0.765110160655909, 'ActionCard': 0.24489214269729334, 'Player': 0.2282741324495544}, 'Game': {'ActionCardType': 0.28189102516393894, 'Game': 0.9313286889417757, 'Board': 0.3800276814784601, 'Die': 0.20455448345932425, 'Tile': 0.3356408493114902, 'ActionCard': 0.3068587291396103, 'Player': 0.6622570304271429}, 'Connection': {'ActionCardType': 0.2906075015161558, 'Game': 0.12434748342860033, 'Board':

[('TileO', None),
 ('Game', 'Game'),
 ('Connection', None),
 ('Die', 'Die'),
 ('Deck', None),
 ('abstract Tile', 'Tile'),
 ('abstract ActionCard', 'ActionCard'),
 ('RollDieActionCard', None),
 ('ConnectTilesActionCard', None),
 ('RemoveConnectionActionCard', None),
 ('TeleportActionCard', None),
 ('LoseTurnActionCard', None),
 ('WinTile', None),
 ('ActionTile', None),
 ('NormalTile', None),
 ('Player', 'Player'),
 (None, 'Board')]

In [2660]:
mapping = enum_mapping[0] + cls_mapping[0]

for map in mapping:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

In [2661]:
ref_cls.cls_atr

{'Mode': {'score': 0,
  'type': 'enum',
  'dsl': 'Mode(design, game)',
  'counterpart': None,
  'attributes': {'design': {'score': 0, 'counterpart': None},
   'game': {'score': 0, 'counterpart': None}}},
 'Color': {'score': 0,
  'type': 'enum',
  'dsl': 'Color(red, blue, green, yellow)',
  'counterpart': None,
  'attributes': {'red': {'score': 0, 'counterpart': None},
   'blue': {'score': 0, 'counterpart': None},
   'green': {'score': 0, 'counterpart': None},
   'yellow': {'score': 0, 'counterpart': None}}},
 'TileO': {'score': 0,
  'type': 'regular',
  'dsl': 'TileO()',
  'counterpart': None,
  'attributes': {}},
 'Game': {'score': 1,
  'type': 'regular',
  'dsl': 'Game(int currentConnectionPieces, Mode mode, int spaceConnectionPieces, int numberOfActionCards)',
  'counterpart': 'Game',
  'attributes': {'int currentConnectionPieces': {'score': 0,
    'counterpart': None},
   'Mode mode': {'score': 0, 'counterpart': None},
   'int spaceConnectionPieces': {'score': 0, 'counterpart': Non

## Stage 1.2 Class Mapping (with all info)

In [2662]:
ref_classes = []
stu_classes = []

for cls in ref_cls.cls_name:
  if ref_cls.cls_atr[cls]["counterpart"] == None:
    ref_classes.append(cls)

for cls in stu_cls.cls_name:
  if stu_cls.cls_atr[cls]["counterpart"] == None:
    stu_classes.append(cls)

In [2663]:
# get name embedding
# all info
similarity_all = []
mde_embedding = sgram_mde
threshold = 0.7




# reuse saved embedding result...
# may cause problem
# if os.path.exists(student_dir + "ada_embedding.txt"):
#   pass
  
  # with open(student_dir + "ada_embedding.txt", 'r') as json_file:
  #   dict_sim_all = json.load(json_file)

if False:
  pass

else:
  for index, node in enumerate(ref_cls.cls_name):
    text_1 = get_all_info(index, ref_cls.cls_name, ref_cls.raw_dsl, ref_cls.rel.raw_dsl, True)
    emb_i = get_embedding(text_1)
    mde_pair = []
    for j, stu_node in enumerate(stu_cls.cls_name):
      text_2 = get_all_info(j, stu_cls.cls_name, stu_cls.raw_dsl, stu_cls.rel.raw_dsl, True)
      emb_j = get_embedding(text_2)
      mde_sim = cosine_distance(emb_i, emb_j)
      mde_pair.append(mde_sim)

    # apply third quartile
    similarity_all.append(mde_pair)

  dict_sim_all = {}
  for i in range(len(ref_cls.cls_name)):
    dict_sim_all[ref_cls.cls_name[i]] = {}
    for j in range(len(stu_cls.cls_name)):
      dict_sim_all[ref_cls.cls_name[i]][stu_cls.cls_name[j]] = similarity_all[i][j]





In [2664]:
with open(student_dir + "ada_embedding.txt", 'w') as json_file:
    json.dump(dict_sim_all, json_file)

In [2665]:
student_dir

'results/gpt-4o/yujing/07_Tile/'

In [2666]:
ref_classes

['Mode',
 'Color',
 'TileO',
 'Connection',
 'Deck',
 'RollDieActionCard',
 'ConnectTilesActionCard',
 'RemoveConnectionActionCard',
 'TeleportActionCard',
 'LoseTurnActionCard',
 'WinTile',
 'ActionTile',
 'NormalTile']

In [2667]:
stu_classes

['ActionCardType', 'Board']

In [2668]:
dict_sim_all

{'Mode': {'ActionCardType': 0.777702502966393,
  'Game': 0.8244502302111426,
  'Board': 0.8003124058499295,
  'Die': 0.7945335803096161,
  'Tile': 0.7679639495332788,
  'ActionCard': 0.7682451880242392,
  'Player': 0.7812574454142899},
 'Color': {'ActionCardType': 0.7464428623524403,
  'Game': 0.7380739890702702,
  'Board': 0.7777599471988865,
  'Die': 0.7895114818360068,
  'Tile': 0.7533249545145939,
  'ActionCard': 0.7311815658380927,
  'Player': 0.7886492905085641},
 'TileO': {'ActionCardType': 0.7801489808452534,
  'Game': 0.7514772499215094,
  'Board': 0.8033025047718675,
  'Die': 0.8187363198183137,
  'Tile': 0.8095944925628566,
  'ActionCard': 0.7313043728830252,
  'Player': 0.7676285370689749},
 'Game': {'ActionCardType': 0.829042243983316,
  'Game': 0.833335317140809,
  'Board': 0.813423808071896,
  'Die': 0.7760971746473532,
  'Tile': 0.8101148037981722,
  'ActionCard': 0.7899889183516379,
  'Player': 0.8094468686876671},
 'Connection': {'ActionCardType': 0.7733959105916384,


In [2669]:
mapping = match_classes(ref_classes, stu_classes, dict_sim_all, 0.8)
mapping[0]

{'Mode': {'ActionCardType': 0.777702502966393, 'Game': 0.8244502302111426, 'Board': 0.8003124058499295, 'Die': 0.7945335803096161, 'Tile': 0.7679639495332788, 'ActionCard': 0.7682451880242392, 'Player': 0.7812574454142899}, 'Color': {'ActionCardType': 0.7464428623524403, 'Game': 0.7380739890702702, 'Board': 0.7777599471988865, 'Die': 0.7895114818360068, 'Tile': 0.7533249545145939, 'ActionCard': 0.7311815658380927, 'Player': 0.7886492905085641}, 'TileO': {'ActionCardType': 0.7801489808452534, 'Game': 0.7514772499215094, 'Board': 0.8033025047718675, 'Die': 0.8187363198183137, 'Tile': 0.8095944925628566, 'ActionCard': 0.7313043728830252, 'Player': 0.7676285370689749}, 'Game': {'ActionCardType': 0.829042243983316, 'Game': 0.833335317140809, 'Board': 0.813423808071896, 'Die': 0.7760971746473532, 'Tile': 0.8101148037981722, 'ActionCard': 0.7899889183516379, 'Player': 0.8094468686876671}, 'Connection': {'ActionCardType': 0.7733959105916384, 'Game': 0.7625562378684552, 'Board': 0.7729449443907

[('Mode', None),
 ('Color', None),
 ('TileO', None),
 ('Connection', None),
 ('Deck', 'Board'),
 ('RollDieActionCard', None),
 ('ConnectTilesActionCard', None),
 ('RemoveConnectionActionCard', None),
 ('TeleportActionCard', None),
 ('LoseTurnActionCard', 'ActionCardType'),
 ('WinTile', None),
 ('ActionTile', None),
 ('NormalTile', None)]

In [2670]:
for map in mapping[0]:
  if map[0] and map[1]:
    ref_cls.cls_atr[map[0]]['score'] = 1
    ref_cls.cls_atr[map[0]]['counterpart'] = map[1]

    stu_cls.cls_atr[map[1]]['score'] = 1
    stu_cls.cls_atr[map[1]]['counterpart'] = map[0]

    if ref_cls.cls_atr[map[0]]["type"] !=  stu_cls.cls_atr[map[1]]["type"]:
      # type mismatch
      ref_cls.cls_atr[map[0]]['score'] = 0.5
      stu_cls.cls_atr[map[1]]["score"] = 0.5

## Stage 2.1 Attribute mapping

In [2671]:
def get_cosine_disntance(text1, text2, embedding):
    avg = np.sum(embedding[text1], axis=0)
    emb_i = avg / len(text1)

    avg = np.sum(embedding[text2], axis=0)
    emb_j = avg / len(text2)

    return np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))

In [2672]:
def create_cosine_similarity_dict(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  dict_attr = {}
  for i in range(len(attr_1)):
    dict_attr[attr_1[i]] = {}
    for j in range(len(attr_2)):
      dict_attr[attr_1[i]][attr_2[j]] = similarities[i][j]

  return dict_attr

In [2673]:
def match_attributes(raw_1, raw_2, dict_attr, threshold = 0.5):
  # map attributes lists raw_1 and raw_2
  # >>> mapAttributes(['deviceStatus'], ['id'], dic)[0]
  def node_subst_cost_attr(node1, node2):
    # threshod as 0.45
    if dict_attr[node1["name"]][node2["name"]] < threshold:
      return 3
    else:
      return 1-dict_attr[node1["name"]][node2["name"]]

  G_att_1 = nx.Graph()
  G_att_2 = nx.Graph()
  for node in raw_1:
    G_att_1.add_node(node, name = node)

  for node in raw_2:
    G_att_2.add_node(node, name = node)

  for v in nx.optimize_edit_paths(G_att_1, G_att_2, node_subst_cost=node_subst_cost_attr, edge_match=None, timeout = 20):
      minv = v
  # minv

  return minv

In [2674]:
def check_attributes_type(attr_1, attr_2, ref_dict, stu_dict):
  # given two attribute, compare whether their type can match
  attr_1 = attr_1.split() # from reference solution do not need to check it
  attr_2 = attr_2.split()

  # if enumerate literal and attributes
  # if len(attr_1) != len(attr_2):
  #   return 0.5, 0.5

  # if an attributes use a regular class as its type
  if len(attr_2) == 2:
    t = attr_2[0].strip()
    cls = stu_dict.get(t, None)
    if cls is not None and (stu_dict[t]['type'] == "regular" or stu_dict[t]['type'] == "abstract"):
      return 0.5, 0.5
    
  if len(attr_1) == 2 or len(attr_2) == 2:
    t = attr_1[0].strip()
    cls = ref_dict.get(t, None)
    # print(cls)
    t_2 = attr_2[0].strip()
    cls_2 = stu_dict.get(t_2, None)   
    # print(cls_2) 
    if cls is not None:
      if ref_dict[t]['counterpart'] != t_2:
        return 0.5, 0.5
    if cls_2 is not None:
      if stu_dict[t_2]['counterpart'] != t:
        return 0.5, 0.5
    
  return 1,1

In [2675]:
x =check_attributes_type(('CommandStatus commandStatus', 'ControllerStatus status')[0], ('CommandStatus commandStatus', 'ControllerStatus status')[1], ref_cls.cls_atr, stu_cls.cls_atr)
x

(1, 1)

### Stage 2.1.1 Attribute <-> Attribute (mapped classes)

In [2676]:
# map classes
pairs = []

# get matched pairs
for key in ref_cls.cls_atr:
  if ref_cls.cls_atr[key]["counterpart"] != None:
    pair = [key, ref_cls.cls_atr[key]["counterpart"]]
    pairs.append(pair)

for pair in pairs:
  if pair[0] and pair[1]:
    print(pair)

    # map attributes:
    # matched_ref = set()
    # matched_attr = set()

    raw_1 = []
    for attributes in ref_cls.cls_atr[pair[0]]["attributes"]:
      raw_1.append(attributes)

    raw_2 = []
    for attributes in stu_cls.cls_atr[pair[1]]["attributes"]:
      raw_2.append(attributes)

    cos_dict = create_cosine_similarity_dict(raw_1, raw_2, sgram_mde)
    mappings = match_attributes(raw_1, raw_2, cos_dict)[0]
    print("mapping", mappings)

    for mapping in mappings:
      if mapping[0] != None and mapping[1] != None:
        scores = check_attributes_type(mapping[0], mapping[1], ref_cls.cls_atr, stu_cls.cls_atr)
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]['score'] = scores[0]
        ref_cls.cls_atr[pair[0]]["attributes"][mapping[0]]["counterpart"] = (mapping[1], pair[1])
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]['score'] = scores[1]
        stu_cls.cls_atr[pair[1]]["attributes"][mapping[1]]["counterpart"] =  (mapping[0], pair[0])

    print("="*20)

['Game', 'Game']
mapping [('int currentConnectionPieces', 'string status'), ('Mode mode', None), ('int spaceConnectionPieces', None), ('int numberOfActionCards', 'int currentPlayerIndex'), (None, 'boolean isGameOver'), (None, 'string version'), (None, 'string designerName')]
['Die', 'Die']
mapping [(None, 'int sides')]
['Deck', 'Board']
mapping [(None, 'int width'), (None, 'int height')]
['abstract Tile', 'Tile']
mapping [('int x', 'int turnsRemaining'), ('int y', None), ('boolean hasBeenVisited', 'boolean isVisited'), (None, 'boolean isHidden'), (None, 'boolean isActionTile')]
['abstract ActionCard', 'ActionCard']
mapping [('string instructions', 'string description'), (None, 'ActionCardType type')]
['LoseTurnActionCard', 'ActionCardType']
mapping [(None, 'RollExtraTurn'), (None, 'ConnectTiles'), (None, 'RemoveConnection'), (None, 'MoveToTile'), (None, 'LoseTurn')]
['Player', 'Player']
mapping [('int number', 'string name'), ('int turnsUntilActive', 'int currentPosition'), ('Color col

### Stage 2.1.2 Attribute <-> Attribute (between any classes)

In [2677]:
def create_cosine_similarity_list(attr_1, attr_2, embedding):
  # attr_1: list[list] split camel case into separated case
  # attr_2: list[list] split camel case into separated case
  # embedding:
  # raw_1: list[str] camcel case attributes
  # raw_2: list[str] camcel case attributes

  # create cosine distance between two attributes with embeeding

  # >>> dic = createCosineDistance(['deviceStatus', 'deviceId'], ['id'], sgram_mde)
  similarities = []
  for att in attr_1:
    emb_i = get_mde_embedding(att, embedding)
    pair = []
    for attribute in attr_2:
      emb_j = get_mde_embedding(attribute, embedding)

      sim = np.dot(emb_i, emb_j) / (np.linalg.norm(emb_i) * np.linalg.norm(emb_j))
      pair.append(sim)

    similarities.append(pair)

  return similarities

In [2678]:
def combine_two_dict(list_1, list_2, atr_1, atr_2, cls_1, cls_2, weight_1 = 0.9, weight_2=0.1):
  # list_1: similarity between cls
  # list_2: cos similarity between attributes
  if weight_1 + weight_2 != 1:
    raise ValueError("weight_1 + weight_2 != 1")
  result = {}
  for i in range(len(atr_1)):
    atr_r = atr_1[i]
    cls_r = cls_1[i]
    tmp_ref = (atr_r, cls_r)
    result[tmp_ref] = {}
    for j in range(len(atr_2)):
      atr_s = atr_2[j]
      cls_s = cls_2[j]
      tmp_stu = (atr_s, cls_s)

      result[tmp_ref][tmp_stu] = weight_1 * list_1[i][j] + weight_2 * list_2[i][j]

  return result


In [2679]:
# attr between any classes

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get attributes on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)


    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 0.5)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 0.5)
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] =  mapping[0]

    if ref_cls.cls_atr[mapping[0][1]]['counterpart'] == mapping[1][1]:
      ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = min(scores[0], 1)

      stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = min(scores[1], 1)
print("="*20)

(('design', 'Mode'), None)
(('game', 'Mode'), ('boolean isGameOver', 'Game'))
(('red', 'Color'), None)
(('blue', 'Color'), None)
(('green', 'Color'), None)
(('yellow', 'Color'), None)
(('Mode mode', 'Game'), None)
(('int spaceConnectionPieces', 'Game'), ('int sides', 'Die'))
(('int y', 'abstract Tile'), ('int height', 'Board'))
(('int inactivityPeriod', 'ActionTile'), ('int width', 'Board'))
(('int turnsUntilActive', 'ActionTile'), ('boolean isActionTile', 'Tile'))
(None, ('RollExtraTurn', 'ActionCardType'))
(None, ('ConnectTiles', 'ActionCardType'))
(None, ('RemoveConnection', 'ActionCardType'))
(None, ('MoveToTile', 'ActionCardType'))
(None, ('LoseTurn', 'ActionCardType'))
(None, ('string version', 'Game'))
(None, ('string designerName', 'Game'))
(None, ('boolean isHidden', 'Tile'))
(None, ('ActionCardType type', 'ActionCard'))


## Stage 2.2.1 Attribute mapping atr -> cls

In [2680]:
#  get non-mapped attributes on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get attributes on instrucotr sides
for cls in ref_cls.cls_atr:
  for attributes in ref_cls.cls_atr[cls]["attributes"]:
    if ref_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_1.append(attributes)
      ref_source.append(cls)
      tup_r.append((attributes, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
    if stu_cls.cls_atr[cls]["counterpart"] == None:
      raw_2.append(cls)
      stu_source.append(cls)
      tup_s.append((cls, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["attributes"][mapping[0][0]]["counterpart"] = (None, mapping[1][1])

    stu_cls.cls_atr[mapping[1][1]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["counterpart"] =  mapping[0]

print("="*20)


(('design', 'Mode'), None)
(('red', 'Color'), None)
(('blue', 'Color'), None)
(('green', 'Color'), None)
(('yellow', 'Color'), None)
(('Mode mode', 'Game'), None)


## Stage 2.2.2 Attribute mapping cls -> atr

In [2681]:
#  get non-mapped cls on instrucotr side

raw_1 = []
ref_source = []
tup_r = []
# get class on instrucotr sides
for cls in ref_cls.cls_atr:
    if ref_cls.cls_atr[cls]["counterpart"] == None:
      raw_1.append(cls)
      ref_source.append(cls)
      tup_r.append((cls, cls))

raw_2 = []
stu_source = []
tup_s = []
# get un-mapped class on student sides
for cls in stu_cls.cls_atr:
  for attributes in stu_cls.cls_atr[cls]["attributes"]:
    if stu_cls.cls_atr[cls]["attributes"][attributes]["counterpart"] == None:
      raw_2.append(attributes)
      stu_source.append(cls)
      tup_s.append((attributes, cls))

list_1 = create_cosine_similarity_list(raw_1, raw_2, sgram_mde)
list_2 = create_cosine_similarity_list(ref_source, stu_source, sgram_mde)
combined = combine_two_dict(list_1, list_2, raw_1, raw_2, ref_source, stu_source)

mappings = match_attributes(tup_r, tup_s, combined)[0]

for mapping in mappings:
  print(mapping)
  if mapping[0] != None and mapping[1] != None:
    # scores = check_attributes_type(mapping[0][0], mapping[1][0], ref_cls.cls_atr, stu_cls.cls_atr)
    ref_cls.cls_atr[mapping[0][1]]['score'] = 0.5
    ref_cls.cls_atr[mapping[0][1]]["counterpart"] = mapping[1]

    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]['score'] = 0.5
    stu_cls.cls_atr[mapping[1][1]]["attributes"][mapping[1][0]]["counterpart"] = (None, mapping[0][1])

print("="*20)


(('Mode', 'Mode'), None)
(('Color', 'Color'), None)
(('TileO', 'TileO'), None)
(('Connection', 'Connection'), None)
(('RollDieActionCard', 'RollDieActionCard'), ('RollExtraTurn', 'ActionCardType'))
(('ConnectTilesActionCard', 'ConnectTilesActionCard'), ('ConnectTiles', 'ActionCardType'))
(('RemoveConnectionActionCard', 'RemoveConnectionActionCard'), ('RemoveConnection', 'ActionCardType'))
(('TeleportActionCard', 'TeleportActionCard'), ('ActionCardType type', 'ActionCard'))
(('WinTile', 'WinTile'), None)
(('ActionTile', 'ActionTile'), ('MoveToTile', 'ActionCardType'))
(('NormalTile', 'NormalTile'), None)
(None, ('LoseTurn', 'ActionCardType'))
(None, ('string version', 'Game'))
(None, ('string designerName', 'Game'))
(None, ('boolean isHidden', 'Tile'))


## Stage 3: Relationship mapping

In [2682]:
def check_relations_classes(ref_index, stu_index, ref_elements, stu_elements):
    # ref_index: index of the element in the dsl
    # stu_index: index of the element in the dsl
    # ref_elements: list[str] elements in the dsl
    # stu_elements: list[str] elements in the dsl

    ref_class_1 = ref_elements[ref_index]
    # check whether this is an abstract class
    dict_keys = list(ref_attributes.keys())
    if not ref_class_1 in dict_keys:
      abstract = "abstract " + ref_class_1
      ref_class_1 = abstract

    true_pair = ref_attributes[ref_class_1]["counterpart"]
    if isinstance(true_pair, tuple):
      boolean_1 = False
    elif true_pair == None:
      boolean_1 = False
    else:
      tmp = true_pair.replace("abstract ", "")
      boolean_1 = stu_elements[stu_index] == tmp
    return boolean_1

#### Compare rel

In [2683]:

def compare_edges(ref_e, stu_e):
  # compare if two relationships match
  # return bool, score, score

  ref_elements = ref_e.split()
  stu_elements = stu_e.split()
  # print(ref_elements, stu_elements)

  # exact match first
  # length must match
  n_ref = len(ref_elements)
  n_stu = len(stu_elements)
  if n_ref == n_stu and n_ref == 5:
    # all element must match
    boolean_1 = check_relations_classes(1,1,ref_elements,stu_elements )

    boolean_4 = check_relations_classes(4,4,ref_elements,stu_elements )

    # type
    boolean_2 = ref_elements[2] == stu_elements[2]

    # association
    boolean_0 = ref_elements[0] == stu_elements[0]

    # association
    boolean_3 = ref_elements[3] == stu_elements[3]

    if boolean_0 and boolean_1 and boolean_2 and boolean_3 and boolean_4:
      print("exact match success")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    boolean_1_4 = False
    boolean_4_1 = False
    boolean_1_4 = check_relations_classes(1,4,ref_elements,stu_elements )
    boolean_4_1 = check_relations_classes(4,1,ref_elements,stu_elements )

    # check if the relationship is fliped, this only works for associate, not containment
    if  "associate" in ref_elements[2] and "associate" in stu_elements[2]:
      # association
      boolean_0 = ref_elements[0] == stu_elements[3]

      # association
      boolean_3 = ref_elements[3] == stu_elements[0]

    if boolean_0 and boolean_1_4 and boolean_3 and boolean_4_1 and boolean_2:
      print("match success, flipped associate")
      print(ref_e, "|||", stu_e)
      return True, 1, 1

    # check if relationship is partially match
    if  (boolean_1 and boolean_4) or (boolean_1_4 and boolean_4_1):
      print("match partially success")
      print(ref_e, "|||", stu_e)
      return True, 0.5, 0.5

  if n_ref == n_stu and n_ref == 3:
    # all element must match
    boolean_0 = check_relations_classes(0,0,ref_elements,stu_elements )
    boolean_2 = check_relations_classes(2,2,ref_elements,stu_elements )

    # type
    boolean_1 = ref_elements[1] == stu_elements[1]

    if boolean_0 and boolean_1 and boolean_2:
      print("match success")
      print(ref_e, "|||", stu_e)
      # ref_dict[ref_e]["score"] = 1
      # ref_dict[ref_e]["counterpart"] = stu_e

      # stu_dict[stu_e]["score"] = 1
      # stu_dict[stu_e]["counterpart"] = ref_e
      return True, 1, 1


  return False,0, 0

In [2684]:

for i, ref_tmp in enumerate(edges[0].raw_dsl):
  matchings = []
  for j, stu_tmp in enumerate(edges[1].raw_dsl):
    if edges[0].rels[i]["counterpart"] is None and edges[1].rels[j]["counterpart"] is None:
      result = compare_edges(ref_tmp, stu_tmp)
      if result[0]:
        matching = (i, j, result[1], result[2])
        matchings.append(matching)
        # matched_stu_edges.add(j)
        # matched_ref_edges.add(i)
        # edges[0].rels[i]["score"] = result[1]
        # edges[0].rels[i]["counterpart"] = stu_tmp

        # edges[1].rels[j]["score"] = result[2]
        # edges[1].rels[j]["counterpart"] = ref_tmp
  print(matchings)
  if len(matchings) > 0:
    print()
    highest_score = 0
    matching = None

    for m in matchings:
      if m[2] > highest_score:
        matching = m
        highest_score = m[2]
    j_j = matching[1]
    i_i = matching[0]
    
    edges[0].rels[i_i]["score"] = m[2]
    edges[0].rels[i_i]["counterpart"] = edges[1].raw_dsl[j_j]

    edges[1].rels[j_j]["score"] = m[3]
    edges[1].rels[j_j]["counterpart"] = edges[0].raw_dsl[i_i]



## Result: Precision/Recall/F1

### save matching to disk

In [2685]:
if not os.path.exists(student_out_dir):
    os.makedirs(student_out_dir)

In [2686]:
# with open( student_out_dir + 'matching.pkl', 'wb') as outp:
#     pickle.dump(grader, outp, pickle.HIGHEST_PROTOCOL)

In [2687]:
student_out_dir

'results/gpt-4o/yujing/07_Tile/result/'

In [2688]:


# as requested in comment

with open(student_out_dir + 'solution_matching.txt', 'w') as file:
     file.write(json.dumps(grader.ref.cls_atr))


with open(student_out_dir + 'student_matching.txt', 'w') as file:
     file.write(json.dumps(grader.stu.cls_atr))

with open(student_out_dir + 'solution_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.ref.rel.rels))


with open(student_out_dir + 'student_matching_relationship.txt', 'w') as file:
     file.write(json.dumps(grader.stu.rel.rels))

In [2689]:
student_out_dir

'results/gpt-4o/yujing/07_Tile/result/'

###### to local

In [2690]:
# # # be careful...

# with open(student_dir + "human_eval/" + 'ref_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.ref.cls_atr))


# with open(student_dir + "human_eval/" + 'stu_meta_cls.py', 'w') as file:
#     file.write(json.dumps(grader.stu.cls_atr))

# with open(student_dir + "human_eval/" + 'ref_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.ref.rel.rels))


# with open(student_dir + "human_eval/" + 'stu_meta_rels.py', 'w') as file:
#     file.write(json.dumps(grader.stu.rel.rels))

In [2691]:
algo_result = {}
algo_result["class"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["attribute"] = {"precision": 0, "recall": 0, "f1":0}
algo_result["relationship"] = {"precision": 0, "recall": 0, "f1":0}

### Class

In [2692]:
# recell
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  count += 1
  recall += ref_attributes[key]["score"]
print(count, recall)

if count != 0:
  algo_result["class"]['recall'] = recall / count

18 8.0


In [2693]:
# precision
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  count += 1
  precision += stu_attributes[key]["score"]

print(count, precision)
if count != 0:
  algo_result["class"]['precision'] = precision / count

if algo_result["class"]['recall']   + algo_result["class"]['precision'] != 0:
  algo_result["class"]['f1'] = 2* (algo_result["class"]['recall']   * algo_result["class"]['precision'] ) / ( algo_result["class"]['recall']   + algo_result["class"]['precision']  )


7 5.5


### Attribute

In [2694]:
# recall
count = 0
recall = 0
for key in list(ref_attributes.keys()):
  attrs = ref_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    recall += ref_attributes[key]["attributes"][att]["score"]
print(count, recall)

if count != 0:
  algo_result["attribute"]['recall'] = recall / count

19 10.0


In [2695]:
count = 0
precision = 0
for key in list(stu_attributes.keys()):
  attrs = stu_attributes[key]["attributes"]

  for att in attrs:
    count += 1
    precision += stu_attributes[key]["attributes"][att]["score"]
print(count, precision)

if count != 0:
  algo_result["attribute"]['precision'] = precision / count

r = algo_result["attribute"]['recall']
p = algo_result["attribute"]['precision']

if r + p != 0:
  algo_result["attribute"]['f1'] = 2* (r * p) / (r + p)


22 12.5


### Relationship

In [2696]:
count = 0
recall = 0
for tmp in edges[0].rels:
  attrs = tmp["score"]

  count += 1
  recall += attrs
print(count, recall)

if count != 0:
  algo_result["relationship"]['recall'] = recall / count

0 0


In [2697]:
count = 0
precision = 0
for tmp in edges[1].rels:
  attrs = tmp["score"]

  count += 1
  precision += attrs
print(count, precision)

if count!= 0:
  algo_result["relationship"]['precision'] = precision / count
r = algo_result["relationship"]['recall']
p = algo_result["relationship"]['precision']

if r + p != 0:
  algo_result["relationship"]['f1'] = 2* (r * p ) / ( r + p )


0 0


In [2698]:

# as requested in comment

with open(student_out_dir + 'algo_result.txt', 'w') as file:
     file.write(json.dumps(algo_result))